In [1]:
# Data handling
import numpy as np                   # NumPy is used for numerical operations, especially on arrays and matrices.
import pandas as pd                 # Pandas is used for data manipulation and analysis using dataframes and series.

# Visualization
import seaborn as sns               # Seaborn is a statistical data visualization library built on top of matplotlib.
import matplotlib.pyplot as plt     # Matplotlib is used to create static, animated, and interactive plots.

# Preprocessing
from sklearn.preprocessing import StandardScaler  
# StandardScaler standardizes features by removing the mean and scaling to unit variance. 
# This is important before applying cosine similarity because it ensures each feature contributes equally.

# Similarity calculation
from sklearn.metrics.pairwise import cosine_similarity
# cosine_similarity computes the cosine similarity between rows in matrices.
# Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space.


In [3]:
# Load datasets
books = pd.read_csv("Books.csv", encoding='latin-1', low_memory=False)
# Reads the Books.csv file into a pandas DataFrame called `books`.
# encoding='latin-1' is used because the file might contain special characters not supported by the default UTF-8.
# low_memory=False avoids breaking up the file into small chunks when loading (useful for mixed data types).

ratings = pd.read_csv("Ratings.csv", encoding='latin-1', low_memory=False)
# Reads the Ratings.csv file into a DataFrame called `ratings`, with the same encoding and memory handling as above.

users = pd.read_csv("Users.csv", encoding='latin-1', low_memory=False)
# Loads the Users.csv file into the DataFrame `users`.

# Display basic info
print("📘 Books Dataset Info:")
books.info()
# Displays information about the 'books' DataFrame:
# - Number of entries (rows)
# - Number of columns
# - Data types of each column
# - Non-null value counts

print("\n⭐ Ratings Dataset Info:")
ratings.info()
# Shows similar details for the 'ratings' DataFrame

print("\n👤 Users Dataset Info:")
users.info()
# Shows similar details for the 'users' DataFrame

# These info() calls help identify:
# - Missing/null values
# - Memory usage
# - Whether certain columns have unexpected types (e.g., numbers stored as strings)


📘 Books Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB

⭐ Ratings Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  

In [4]:
# Drop duplicates based on 'Book-Title'
new_book = books.drop_duplicates(subset='Book-Title', keep='first')
# This line removes duplicate entries from the 'books' DataFrame based on the 'Book-Title' column.
# 'subset' specifies the column to check for duplicates.
# 'keep="first"' means it keeps the first occurrence and drops the rest.
# The cleaned result is stored in a new DataFrame called 'new_book'.

# Show how many rows were dropped
print(f"Original books count: {books.shape[0]}")
# This prints the total number of rows (books) in the original 'books' DataFrame before removing duplicates.

print(f"Books after removing duplicates: {new_book.shape[0]}")
# This prints the total number of rows (books) in the new DataFrame after duplicates have been removed.
# Helps us understand how many entries were considered duplicates and dropped.
#[0] for rows 
#[1] for coloums 


Original books count: 271360
Books after removing duplicates: 242135


In [5]:
# Merge ratings with cleaned book data
ratings_with_name = ratings.merge(new_book, on='ISBN')
# This merges the 'ratings' DataFrame with the cleaned 'new_book' DataFrame.
# The merge is done on the 'ISBN' column, which uniquely identifies books.
# This way, each rating now has the book title and metadata (e.g., author, publisher) attached to it.

# Drop unnecessary columns
columns_to_drop = ['ISBN', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']
# This list contains columns that are not needed for recommendation:
# - 'ISBN' is no longer needed since we now use book titles.
# - The image URLs (small, medium, large) are not used in our model logic.

ratings_with_name.drop(columns=columns_to_drop, inplace=True)
# This drops the above-listed columns from the merged DataFrame.
# inplace=True modifies the DataFrame directly without returning a copy.

# Show the updated dataframe
print("Merged DataFrame shape:", ratings_with_name.shape)
# Prints the shape of the new DataFrame (rows, columns) after dropping columns.

ratings_with_name.head()
# Displays the first 5 rows of the DataFrame to quickly verify the merged and cleaned data.


Merged DataFrame shape: (883079, 6)


,User-ID,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,276725,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,276729,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press
2,276729,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press
3,276746,0,Lightning,Dean R. Koontz,1996,Berkley Publishing Group
4,276746,0,Manhattan Hunt Club,JOHN SAUL,2002,Ballantine Books


In [6]:
# Merge ratings with user data
users_ratings_matrix = ratings_with_name.merge(users, on='User-ID')
# This merges the 'ratings_with_name' DataFrame (which includes book info) with the 'users' DataFrame.
# The merge is done on 'User-ID', allowing us to include user-specific data (like location and age) in each rating.

# Drop non-relevant columns
columns_to_drop = ['Location', 'Age']
# These columns are user metadata that are not required for building the recommendation engine.
# They are not used in calculating similarity or predictions.

users_ratings_matrix.drop(columns=columns_to_drop, inplace=True)
# Removes 'Location' and 'Age' from the merged DataFrame.
# inplace=True applies the change directly to the existing DataFrame.

# Show the cleaned merged data
print("User-Ratings Matrix shape:", users_ratings_matrix.shape)
# Prints the shape (rows and columns) of the final cleaned DataFrame that contains ratings with book and user info.

users_ratings_matrix.head()
# Displays the first 5 rows of the cleaned DataFrame to visually inspect the structure and content.


User-Ratings Matrix shape: (883079, 6)


,User-ID,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,276725,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,276729,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press
2,276729,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press
3,276746,0,Lightning,Dean R. Koontz,1996,Berkley Publishing Group
4,276746,0,Manhattan Hunt Club,JOHN SAUL,2002,Ballantine Books


In [7]:
# Check for null values in the merged user-ratings matrix
print("Null values before cleaning:")
print(users_ratings_matrix.isnull().sum())
# This checks for any missing (null/NaN) values in each column of the 'users_ratings_matrix' DataFrame.
# .isnull() returns a DataFrame of True/False for nulls, and .sum() counts them per column.
# Useful to know where and how many nulls exist before cleaning.

# Drop rows with any null values
users_ratings_matrix.dropna(inplace=True)
# Removes all rows that contain at least one null value.
# inplace=True ensures the DataFrame is updated directly without creating a copy.
# This helps ensure cleaner data for model training and avoids errors later on.

# Confirm all nulls are removed
print("\nNull values after cleaning:")
print(users_ratings_matrix.isnull().sum())
# Rechecks for null values to confirm that the DataFrame is now free of them.
# If all columns return 0, it means no missing data remains.


Null values before cleaning:
User-ID                0
Book-Rating            0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              1
dtype: int64

Null values after cleaning:
User-ID                0
Book-Rating            0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
dtype: int64


In [8]:
# Step 1: Filter users who have rated more than 100 books
user_rating_counts = users_ratings_matrix['User-ID'].value_counts()
# Counts how many times each user appears in the dataset (i.e., how many books each user has rated).
# Returns a Series with 'User-ID' as the index and their rating counts as values.

active_users = user_rating_counts[user_rating_counts > 100].index
# Filters the users who have rated more than 100 books.
# .index gives the user IDs that meet this condition (active users).

filtered_users_ratings = users_ratings_matrix[users_ratings_matrix['User-ID'].isin(active_users)]
# Keeps only the rows in which the 'User-ID' is in the list of active users.
# Result: a smaller DataFrame with only ratings by frequent users.

# Step 2: Filter books that have received at least 50 ratings
book_rating_counts = filtered_users_ratings['Book-Title'].value_counts()
# Counts how many times each book title appears in the filtered dataset (i.e., how often each book has been rated).

popular_books = book_rating_counts[book_rating_counts >= 50].index
# Filters the books that have been rated at least 50 times.
# .index gives the titles of those popular books.

filtered_users_ratings = filtered_users_ratings[filtered_users_ratings['Book-Title'].isin(popular_books)]
# Further filters the DataFrame to keep only the ratings of the popular books.

# Final shape after filtering
print("Filtered Data shape:", filtered_users_ratings.shape)
# Prints the number of rows and columns in the final filtered DataFrame.

filtered_users_ratings.head()
# Displays the first 5 rows of the filtered dataset.
# This will show data with users who rate frequently and books that are widely rated.


Filtered Data shape: (51997, 6)


,User-ID,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
990,277427,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
1002,277427,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial
1004,277427,0,Bel Canto: A Novel,Ann Patchett,2002,Perennial
1007,277427,9,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch
1015,277427,0,The Tao of Pooh,Benjamin Hoff,1983,Penguin Books


In [9]:
# 📊 Create pivot table (user-item matrix)
user_item_matrix = filtered_users_ratings.pivot_table(
    index='User-ID',            # 🔑 Rows = individual users (each row represents a unique user)
    columns='Book-Title',       # 📚 Columns = book titles (each column is a different book)
    values='Book-Rating'        # 🌟 Cell values = the rating a user gave to a book
)
# 🎯 This transforms the long-format DataFrame into a wide matrix where users are rows,
#     books are columns, and each cell shows the rating the user gave to that book.
#     If the user hasn't rated a book, the cell will be NaN (missing).

# 🧹 Fill missing values with 0 (indicating no rating)
user_item_matrix.fillna(0, inplace=True)
# ❗ Replaces NaN with 0 because similarity algorithms (like cosine similarity) require
#    numerical values. Zero means the user didn’t rate that book.

# 🧾 Check the shape of the user-item matrix
print("User-Item Matrix Shape:", user_item_matrix.shape)
# ✅ Displays the matrix dimensions — (number of users, number of books)

# 👁️ Preview the first 5 rows of the matrix
user_item_matrix.head()
# 📄 Shows a few rows to verify that the pivot and fill were successful.


User-Item Matrix Shape: (1391, 620)


Book-Title,1984,1st to Die: A Novel,2010: Odyssey Two,2nd Chance,4 Blondes,A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,A Case of Need,"A Child Called \It\"": One Child's Courage to Survive""",A Civil Action,A Cry In The Night,...,Wicked: The Life and Times of the Wicked Witch of the West,Wifey,Wild Animus,Without Remorse,"Word Freak: Heartbreak, Triumph, Genius, and Obsession in the World of Competitive Scrabble Players",Year of Wonders,You Belong To Me,Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,Zoya,"\O\"" Is for Outlaw"""
User-ID,,,,,,,,,,,,,,,,,,,,,
254,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0
1435,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# ✅ Step 1: Standardize the user-item matrix (row-wise i.e., by user)
scaler = StandardScaler()                  # 🔧 Create a scaler object for standardizing data
user_item_matrix_scaled = scaler.fit_transform(user_item_matrix)
# 📉 StandardScaler subtracts the mean and divides by standard deviation for each feature (column),
# but since we apply it to the matrix directly, it standardizes row-wise by default here.
# 💡 Why? This helps eliminate bias caused by users who rate too high or too low on average.

# ✅ Step 2: Transpose matrix so we compare books (item-item)
# 📍 Now rows = books, columns = users
item_user_matrix = user_item_matrix_scaled.T
# 🔁 Transposing is crucial because we want to calculate similarity **between books**
# based on how users rated them — i.e., compare rating patterns across users for each book.

# ✅ Step 3: Compute cosine similarity between books
similarity_matrix = cosine_similarity(item_user_matrix)
# 📐 Cosine similarity gives a score between -1 and 1 indicating how similar two books are,
# based on user ratings — 1 means highly similar, 0 means orthogonal (unrelated), -1 means opposite.

# ✅ Step 4: Convert to DataFrame for easy lookup
similarity_df = pd.DataFrame(similarity_matrix, 
                             index=user_item_matrix.columns,    # 📘 Book titles as row index
                             columns=user_item_matrix.columns)  # 📘 Book titles as column names
# 📊 This DataFrame allows you to easily query: “How similar is Book A to Book B?”

# 👁️ Preview the similarity matrix
similarity_df.head()
# Shows the top rows of the item-item similarity matrix — values close to 1 mean high similarity.


Book-Title,1984,1st to Die: A Novel,2010: Odyssey Two,2nd Chance,4 Blondes,A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,A Case of Need,"A Child Called \It\"": One Child's Courage to Survive""",A Civil Action,A Cry In The Night,...,Wicked: The Life and Times of the Wicked Witch of the West,Wifey,Wild Animus,Without Remorse,"Word Freak: Heartbreak, Triumph, Genius, and Obsession in the World of Competitive Scrabble Players",Year of Wonders,You Belong To Me,Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,Zoya,"\O\"" Is for Outlaw"""
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,1.000000,0.028839,0.094383,-0.018328,-0.015823,-0.014250,-0.016036,0.037493,0.068979,-0.012564,...,-0.003247,-0.018272,0.009341,-0.013979,0.163970,0.063534,-0.014331,0.061027,-0.010435,-0.016816
1st to Die: A Novel,0.028839,1.000000,0.009294,0.136982,-0.020409,-0.018380,-0.006798,0.039041,0.088839,0.129245,...,0.063267,0.049608,0.034920,0.067275,0.046504,0.024918,0.103243,0.012206,0.110059,0.183378
2010: Odyssey Two,0.094383,0.009294,1.000000,-0.011606,-0.010019,-0.009023,0.076980,-0.014771,0.064663,0.133556,...,0.025970,-0.011570,0.040167,0.080368,0.034297,0.026674,-0.009074,0.090692,-0.006608,-0.010648
2nd Chance,-0.018328,0.136982,-0.011606,1.000000,-0.013834,-0.012459,0.056854,-0.020395,0.165618,0.091150,...,0.084192,0.137899,0.009326,-0.012221,0.020994,0.013973,0.164108,0.021127,0.051793,0.022755
4 Blondes,-0.015823,-0.020409,-0.010019,-0.013834,1.000000,-0.010756,0.073693,-0.017607,-0.007711,-0.009483,...,0.045574,0.069776,-0.005376,-0.010551,0.050265,0.060279,0.043952,-0.012705,-0.007876,-0.012692


In [11]:
from sklearn.metrics.pairwise import cosine_similarity  # 📐 Function to compute cosine similarity between vectors

# ✅ The item-user matrix is already standardized and transposed earlier
# i.e., rows = books, columns = users
# Now we calculate how similar each book is to every other book based on user ratings

item_similarity = cosine_similarity(item_user_matrix)
# 📈 This gives us a square matrix where each cell [i][j] indicates similarity between book i and book j
# Cosine similarity ranges from -1 (opposite) to 1 (very similar)

# ✅ Convert the similarity matrix into a readable DataFrame
item_similarity_df = pd.DataFrame(
    item_similarity,
    index=user_item_matrix.columns,   # 📘 Book titles as row labels
    columns=user_item_matrix.columns # 📘 Book titles as column labels
)
# 🧾 This DataFrame now lets you easily retrieve similarity scores between any two books by their titles

# 👁️ Show a small portion of the similarity matrix to inspect it
print("Item-Item Similarity Matrix (sample):")
item_similarity_df.iloc[:5, :5]
# 🖨️ Displays the top-left 5x5 section of the matrix — showing how the first 5 books relate to each other


Item-Item Similarity Matrix (sample):


Book-Title,1984,1st to Die: A Novel,2010: Odyssey Two,2nd Chance,4 Blondes
Book-Title,,,,,
1984,1.000000,0.028839,0.094383,-0.018328,-0.015823
1st to Die: A Novel,0.028839,1.000000,0.009294,0.136982,-0.020409
2010: Odyssey Two,0.094383,0.009294,1.000000,-0.011606,-0.010019
2nd Chance,-0.018328,0.136982,-0.011606,1.000000,-0.013834
4 Blondes,-0.015823,-0.020409,-0.010019,-0.013834,1.000000


In [12]:
def recommend(book_name, num_recommendations=5):
    # 🛑 Check if the book exists in the similarity matrix
    if book_name not in item_similarity_df.columns:
        return f"❌ Book '{book_name}' not found in similarity matrix."
        # 🔁 This ensures the function doesn't break if the book is missing
    
    # ✅ Step 1: Get similarity scores for the book
    similarity_scores = item_similarity_df[book_name]
    # 📊 This retrieves a Series of similarity values for 'book_name' compared to all other books
    # Index = book titles, Values = similarity score (higher = more similar)

    # ✅ Step 2: Sort books by similarity score (highest first)
    similar_books = similarity_scores.sort_values(ascending=False)
    # 📌 This ensures the most similar books come first in the list

    # ✅ Step 3: Skip the first entry (the book itself), take top N
    top_books = similar_books.iloc[1:num_recommendations+1].index
    # ⚠️ The most similar book will be itself (100% match), so we skip the first entry
    # 🎯 Extract the titles of the top N most similar books

    # ✅ Step 4: Get book details from the new_book DataFrame
    recommendations = []  # 📚 This list will store book info dictionaries

    for title in top_books:
        # 🔎 Find the row in new_book that matches the recommended title
        book_info = new_book[new_book['Book-Title'] == title].iloc[0]
        # ⚠️ .iloc[0] takes the first matching row (in case there are duplicates)

        # 🧾 Create a dictionary with relevant book details
        recommendations.append({
            'Title': title,
            'Author': book_info['Book-Author'],
            'Publisher': book_info['Publisher'],
            'Image-URL': book_info['Image-URL-M']
        })

    return recommendations
    # 📤 Returns a list of dictionaries, each containing:
    # - Title
    # - Author
    # - Publisher
    # - Image URL (medium size)


In [13]:
# Example call to validate the model
recommendations = recommend('1984')  # 🔍 Call the recommend function using the book title '1984'
                                     # ✅ This returns a list of recommended books as dictionaries

# Display the recommendations
for idx, book in enumerate(recommendations, 1):  # 🔁 Loop through the recommendations with index starting at 1
    print(f"\n📘 Recommendation {idx}")           # 🖨️ Print the recommendation number
    print(f"Title     : {book['Title']}")        # 🖨️ Print the title of the recommended book
    print(f"Author    : {book['Author']}")       # 🖨️ Print the author
    print(f"Publisher : {book['Publisher']}")    # 🖨️ Print the publisher
    print(f"Image URL : {book['Image-URL']}")    # 🖨️ Print the image URL (can be used to preview the book cover)



📘 Recommendation 1
Title     : Slaughterhouse Five or the Children's Crusade: A Duty Dance With Death
Author    : Kurt Vonnegut
Publisher : Laurel
Image URL : http://images.amazon.com/images/P/0440180295.01.MZZZZZZZ.jpg

📘 Recommendation 2
Title     : The Handmaid's Tale
Author    : Margaret Atwood
Publisher : Fawcett Books
Image URL : http://images.amazon.com/images/P/0449212602.01.MZZZZZZZ.jpg

📘 Recommendation 3
Title     : The Bonesetter's Daughter
Author    : Amy Tan
Publisher : Putnam Publishing Group
Image URL : http://images.amazon.com/images/P/0399146431.01.MZZZZZZZ.jpg

📘 Recommendation 4
Title     : Word Freak: Heartbreak, Triumph, Genius, and Obsession in the World of Competitive Scrabble Players
Author    : Stefan Fatsis
Publisher : Penguin Books
Image URL : http://images.amazon.com/images/P/0142002267.01.MZZZZZZZ.jpg

📘 Recommendation 5
Title     : Animal Farm
Author    : George Orwell
Publisher : Signet
Image URL : http://images.amazon.com/images/P/0451526341.01.MZZZZZZ

In [14]:
# ✅ Validate the recommendation system using a different book title
recommendations = recommend('The Da Vinci Code')  
# 🔍 Calls the 'recommend' function with the book title 'The Da Vinci Code'.
#     It returns a list of dictionaries, each representing a recommended book.

# 📘 Display each recommended book's details
for idx, book in enumerate(recommendations, 1):  
    # 🔁 Loops over each recommended book using 'enumerate', which gives both index and book data.
    #     'idx' is the serial number starting from 1.
    #     'book' is a dictionary containing book information.

    print(f"\n📘 Recommendation {idx}")  
    # 🖨️ Prints the recommendation number with a book emoji.

    print(f"Title     : {book['Title']}")       
    # 📚 Prints the title of the recommended book.

    print(f"Author    : {book['Author']}")      
    # ✍️ Prints the author of the book.

    print(f"Publisher : {book['Publisher']}")   
    # 🏢 Prints the name of the publisher.

    print(f"Image URL : {book['Image-URL']}")   
    # 🖼️ Prints the medium-sized image URL of the book cover.



📘 Recommendation 1
Title     : Angels &amp; Demons
Author    : Dan Brown
Publisher : Pocket Star
Image URL : http://images.amazon.com/images/P/0671027360.01.MZZZZZZZ.jpg

📘 Recommendation 2
Title     : TickTock
Author    : Dean R. Koontz
Publisher : Ballantine Books
Image URL : http://images.amazon.com/images/P/034538430X.01.MZZZZZZZ.jpg

📘 Recommendation 3
Title     : Touching Evil
Author    : Kay Hooper
Publisher : Bantam Books
Image URL : http://images.amazon.com/images/P/0553583441.01.MZZZZZZZ.jpg

📘 Recommendation 4
Title     : The Blue Nowhere : A Novel
Author    : Jeffery Deaver
Publisher : Pocket
Image URL : http://images.amazon.com/images/P/0671042262.01.MZZZZZZZ.jpg

📘 Recommendation 5
Title     : East of Eden (Oprah's Book Club)
Author    : John Steinbeck
Publisher : Penguin Books
Image URL : http://images.amazon.com/images/P/0142004235.01.MZZZZZZZ.jpg


In [15]:
# ✅ Validate recommendation engine with a new book
recommendations = recommend('To Kill a Mockingbird')  
# 🔍 Call the 'recommend' function with the book title 'To Kill a Mockingbird'.
#     This function returns a list of recommended books (each as a dictionary).

# 📘 Display each recommended book's details
for idx, book in enumerate(recommendations, 1):  
    # 🔁 Loop through each recommended book using enumerate:
    #     'idx' gives the position (starting from 1),
    #     'book' is a dictionary containing recommendation info.

    print(f"\n📘 Recommendation {idx}")  
    # 🖨️ Print the recommendation number with an icon for clarity.

    print(f"Title     : {book['Title']}")       
    # 📚 Print the recommended book's title.

    print(f"Author    : {book['Author']}")      
    # ✍️ Print the author's name.

    print(f"Publisher : {book['Publisher']}")   
    # 🏢 Print the name of the publishing company.

    print(f"Image URL : {book['Image-URL']}")   
    # 🖼️ Print the URL to the book cover image (medium size).



📘 Recommendation 1
Title     : Lord of the Flies
Author    : William Gerald Golding
Publisher : Perigee Trade
Image URL : http://images.amazon.com/images/P/0399501487.01.MZZZZZZZ.jpg

📘 Recommendation 2
Title     : Drowning Ruth
Author    : Christina Schwarz
Publisher : Doubleday
Image URL : http://images.amazon.com/images/P/0385502532.01.MZZZZZZZ.jpg

📘 Recommendation 3
Title     : Animal Farm
Author    : George Orwell
Publisher : Signet
Image URL : http://images.amazon.com/images/P/0451526341.01.MZZZZZZZ.jpg

📘 Recommendation 4
Title     : Mrs Dalloway
Author    : Virginia Woolf
Publisher : Harvest Books
Image URL : http://images.amazon.com/images/P/0156628708.01.MZZZZZZZ.jpg

📘 Recommendation 5
Title     : The Catcher in the Rye
Author    : J.D. Salinger
Publisher : Little, Brown
Image URL : http://images.amazon.com/images/P/0316769487.01.MZZZZZZZ.jpg


In [16]:
recommendations = recommend("Harry Potter and the Chamber of Secrets (Book 2)")
# ✅ Call the `recommend()` function with the title of the book.
#    It returns a list of recommended books (each as a dictionary with details).

for idx, book in enumerate(recommendations, 1):
    # 🔁 Loop over the list of recommended books using `enumerate()`:
    #    `idx` gives the position (starting from 1),
    #    `book` contains details like title, author, publisher, image.

    print(f"\n📘 Recommendation {idx}")
    # 🖨️ Display the recommendation number in a clear format with an emoji.

    print(f"Title     : {book['Title']}")
    # 📚 Print the title of the recommended book.

    print(f"Author    : {book['Author']}")
    # ✍️ Print the author of the recommended book.

    print(f"Publisher : {book['Publisher']}")
    # 🏢 Print the publisher of the recommended book.

    print(f"Image URL : {book['Image-URL']}")
    # 🖼️ Print the medium-sized image URL for the book cover.



📘 Recommendation 1
Title     : Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Author    : J. K. Rowling
Publisher : Arthur A. Levine Books
Image URL : http://images.amazon.com/images/P/059035342X.01.MZZZZZZZ.jpg

📘 Recommendation 2
Title     : F Is for Fugitive (Kinsey Millhone Mysteries (Paperback))
Author    : Sue Grafton
Publisher : Bantam
Image URL : http://images.amazon.com/images/P/0553284789.01.MZZZZZZZ.jpg

📘 Recommendation 3
Title     : Harry Potter and the Goblet of Fire (Book 4)
Author    : J. K. Rowling
Publisher : Scholastic
Image URL : http://images.amazon.com/images/P/0439139597.01.MZZZZZZZ.jpg

📘 Recommendation 4
Title     : Portrait in Death
Author    : Nora Roberts
Publisher : Berkley Publishing Group
Image URL : http://images.amazon.com/images/P/0425189031.01.MZZZZZZZ.jpg

📘 Recommendation 5
Title     : Charlotte's Web (Trophy Newbery)
Author    : E. B. White
Publisher : HarperTrophy
Image URL : http://images.amazon.com/images/P/0064400557.01.MZZZZ